#Import library and dataset

In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv('https://raw.githubusercontent.com/fbhsaid/DAV6300/main/Global%20Baseline%20Data.csv')
df.head()

,Unnamed: 0,Huawei,Xiaomi,Vivo,Apple,Samsung,Sony
0,Olivia,NaN,5,NaN,4.5,3.0,NaN
1,ZJY,4.0,3,NaN,4.0,4.0,NaN
2,LHR,5.0,3,NaN,4.0,NaN,NaN
3,ZHC,4.0,3,3.5,4.5,3.5,2.5
4,ZYC,3.0,4,NaN,3.0,3.0,NaN


#Data Transformations

In [3]:
df.index = ['Olivia', 'ZJY', 'LHR', 'ZHC','ZYC','Tianen']
df.columns =['Col_1', 'Huawei', 'Xiaomi', 'Vivo', 'Apple', 'Samsung', 'Sony']
df1 = df[['Huawei', 'Xiaomi', 'Vivo', 'Apple', 'Samsung', 'Sony']]
df1

,Huawei,Xiaomi,Vivo,Apple,Samsung,Sony
Olivia,NaN,5,NaN,4.5,3.0,NaN
ZJY,4.0,3,NaN,4.0,4.0,NaN
LHR,5.0,3,NaN,4.0,NaN,NaN
ZHC,4.0,3,3.5,4.5,3.5,2.5
ZYC,3.0,4,NaN,3.0,3.0,NaN
Tianen,5.0,4,NaN,4.5,4.0,NaN


In [4]:
df1 = df1.fillna(0)
df1

,Huawei,Xiaomi,Vivo,Apple,Samsung,Sony
Olivia,0.0,5,0.0,4.5,3.0,0.0
ZJY,4.0,3,0.0,4.0,4.0,0.0
LHR,5.0,3,0.0,4.0,0.0,0.0
ZHC,4.0,3,3.5,4.5,3.5,2.5
ZYC,3.0,4,0.0,3.0,3.0,0.0
Tianen,5.0,4,0.0,4.5,4.0,0.0


In [5]:
df1 = df1.T
df1

,Olivia,ZJY,LHR,ZHC,ZYC,Tianen
Huawei,0.0,4.0,5.0,4.0,3.0,5.0
Xiaomi,5.0,3.0,3.0,3.0,4.0,4.0
Vivo,0.0,0.0,0.0,3.5,0.0,0.0
Apple,4.5,4.0,4.0,4.5,3.0,4.5
Samsung,3.0,4.0,0.0,3.5,3.0,4.0
Sony,0.0,0.0,0.0,2.5,0.0,0.0


# Import KNN from scikit learn, set metric to cosine similiarity

In [6]:
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(df1.values)
distances, indices = knn.kneighbors(df1.values, n_neighbors=3)

#Check that we can apply to our dataset successfully

In [7]:
indices

array([[0, 3, 1],
       [1, 3, 4],
       [2, 5, 3],
       [3, 1, 4],
       [4, 3, 1],
       [2, 5, 3]])

In [8]:
distances

array([[0.00000000e+00, 1.11457491e-01, 1.87921506e-01],
       [0.00000000e+00, 2.65006689e-02, 9.42007034e-02],
       [0.00000000e+00, 0.00000000e+00, 5.53886563e-01],
       [3.33066907e-16, 2.65006689e-02, 9.21774535e-02],
       [0.00000000e+00, 9.21774535e-02, 9.42007034e-02],
       [0.00000000e+00, 0.00000000e+00, 5.53886563e-01]])

Define two functions, first is to get phones names based on their index value, the second is to get similiar phones to a particular phone to recommend to someone looking for alternatives to that phone. To do that, we first get the indices of the similiar phones, then we apply the first fuction we defined to get phone names before finally returning them.

In [9]:
#Define function to get phone names based on their index
def getName(arr):
    return [df1.index[index] for index in arr]
#Define function to get similiar phones by getting their indices and applying the getName function we just defined.
def getsimphoneNames (phone):
  index_for_phone = df1.index.tolist().index(phone)
  sim_phones = indices[index_for_phone].tolist()
  if sim_phones[0] == index_for_phone:
    print('similiar phones for '+phone+' is', getName(sim_phones[1:]))
  if sim_phones[0] != index_for_phone:
    print('similiar phones for '+phone+' is', getName(sim_phones[:-1]))
#Apply function to Huawei for similiar phones to Huawei
getsimphoneNames ('Huawei')

similiar phones for Huawei is ['Apple', 'Xiaomi']


##Using LightFM

In [11]:
#Install LightFM first
!pip install LightFM

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.1/310.1 KB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for LightFM: filename=lightfm-1.16-cp38-cp38-linux_x86_64.whl size=917966 sha256=23eb2fad44a8a9952d881a00ee2badbab3bf5a0058681c487e5835f528fc7565
  Stored in directory: /root/.cache/pip/wheels/ec/bb/51/9c487d021c1373b691d13cadca0b65b6852627b1f3f43550fa
Successfully built LightFM


In [37]:
df2 = df1.T
df2

,Huawei,Xiaomi,Vivo,Apple,Samsung,Sony
Olivia,0.0,5.0,0.0,4.5,3.0,0.0
ZJY,4.0,3.0,0.0,4.0,4.0,0.0
LHR,5.0,3.0,0.0,4.0,0.0,0.0
ZHC,4.0,3.0,3.5,4.5,3.5,2.5
ZYC,3.0,4.0,0.0,3.0,3.0,0.0
Tianen,5.0,4.0,0.0,4.5,4.0,0.0


In [38]:
from scipy.sparse import csr_matrix
from scipy import sparse
from lightfm import LightFM
model = LightFM()
data1 = csr_matrix(df2)
data1.toarray()
model.fit(data1)
model.get_user_representations()

(array([0.06518196, 0.08395443, 0.06544697, 0.11744884, 0.08420435,
        0.08300701], dtype=float32),
 array([[-0.01047641, -0.03695489,  0.02684203, -0.00482698,  0.04381001,
          0.03199977, -0.00493728,  0.01251834,  0.01481034,  0.02202491],
        [ 0.041653  , -0.0278026 , -0.04685871, -0.00747679, -0.00347812,
         -0.02260884, -0.02745018, -0.04843992, -0.01683893,  0.01660766],
        [ 0.02060942, -0.0221092 , -0.01105251, -0.0067706 , -0.01332328,
          0.02314905, -0.04788194,  0.01654683, -0.02889526,  0.00048226],
        [-0.02910426,  0.03860835,  0.04396958,  0.00033213, -0.03901605,
          0.04354213,  0.01824321, -0.04479913,  0.00858459, -0.03725356],
        [-0.01852225,  0.00915168, -0.01242727, -0.00706059,  0.00929345,
         -0.03466817, -0.01335071,  0.01656304, -0.00347379, -0.02226132],
        [ 0.03811063,  0.01529623,  0.03096241, -0.02708114,  0.00774089,
          0.00624657, -0.02059204, -0.0391402 ,  0.00447689,  0.01889873]],


In [47]:
phones = np.array(["Huawei", "Xiaomi", "Vivo", "Apple", "Samsung", "Sony"])
def sample_recommendation(model, data, user_ids):

    #number of users and movies in training data
    n_users, n_items = data1.shape

    #generate recommendations for each user we input
    for user_id in user_ids:

        #movies our model predicts they will like
        scores = model.predict(user_id, np.arange(n_items))

        #rank them in order of most liked to least
        top_phones = phones[np.argsort(-scores)]

        print(top_phones)

print('sample phone recommendations for our users are')
print('Olivia')
sample_recommendation(model, data1, [0])
print('ZJY')
sample_recommendation(model, data1, [1])
print('LHR')
sample_recommendation(model, data1, [2])
print('ZHC')
sample_recommendation(model, data1, [3])
print('ZYC')
sample_recommendation(model, data1, [4])
print('Tianen')
sample_recommendation(model, data1, [5])

sample phone recommendations for our users are
Olivia
['Apple' 'Xiaomi' 'Huawei' 'Samsung' 'Vivo' 'Sony']
ZJY
['Xiaomi' 'Apple' 'Samsung' 'Huawei' 'Sony' 'Vivo']
LHR
['Xiaomi' 'Apple' 'Samsung' 'Huawei' 'Sony' 'Vivo']
ZHC
['Xiaomi' 'Apple' 'Huawei' 'Samsung' 'Vivo' 'Sony']
ZYC
['Apple' 'Xiaomi' 'Samsung' 'Huawei' 'Sony' 'Vivo']
Tianen
['Xiaomi' 'Apple' 'Huawei' 'Samsung' 'Sony' 'Vivo']


#My References:
1. https://towardsdatascience.com/item-based-collaborative-filtering-in-python-91f747200fab
2. https://blog.csdn.net/qq_42306860/article/details/97140978
3. https://stackoverflow.com/questions/40323321/creating-a-sparse-matrix-with-lightfm-and-print-predictions